In [1]:
import open3d as o3d
import numpy as np
import os
import glob
import pandas as pd
import pymeshlab as pml
import time
import matplotlib as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
from scipy.stats import wasserstein_distance
from emd import get_emd

In [3]:
wasserstein_distance([0, 1, 3], [5, 6, 8])

5.0

In [4]:
df = pd.read_pickle("./normalized_features_final.pkl")
df.reset_index(drop=True, inplace=True)

In [5]:
df

,class,path,V,S,c,D,R,E,C,A3,D1,D2,D3,D4
1,AircraftBuoyant,./features/AircraftBuoyant\m1337,4.100846,0.214806,-0.021151,-0.431171,1.524371,4.945361,1.790998,"[0.000726, 0.002233, 0.003601, 0.004919, 0.006...","[0.0, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, ...","[0.0, 0.0004, 0.00058, 0.0012, 0.00102, 0.0015...","[2e-06, 5e-06, 1.9e-05, 5.7e-05, 0.000109, 0.0...","[8e-06, 5e-05, 0.000122, 0.000233, 0.000377, 0..."
1,AircraftBuoyant,./features/AircraftBuoyant\m1338,4.728118,0.540607,-0.021151,-0.117928,1.525120,3.309235,1.818866,"[0.000905, 0.002401, 0.003699, 0.004898, 0.006...","[0.0, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, ...","[0.00013, 0.00038, 0.0006, 0.001, 0.00152, 0.0...","[3e-06, 1.6e-05, 4.9e-05, 0.000101, 0.000188, ...","[8e-06, 6.2e-05, 0.00014, 0.000303, 0.00046, 0..."
1,AircraftBuoyant,./features/AircraftBuoyant\m1339,0.087864,0.155798,-0.021151,-0.690660,0.052891,0.829243,0.072976,"[0.002045, 0.005137, 0.007327, 0.009413, 0.011...","[0.0, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, ...","[0.00013, 0.0008, 0.00136, 0.00168, 0.0022, 0....","[6e-06, 1.8e-05, 5.9e-05, 0.000113, 0.00024, 0...","[1.1e-05, 8.1e-05, 0.000217, 0.000387, 0.00064..."
1,AircraftBuoyant,./features/AircraftBuoyant\m1340,0.219291,-0.646455,-0.021151,-0.689441,0.445397,-0.294062,0.667100,"[0.006895, 0.012267, 0.016517, 0.020126, 0.023...","[0.0, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, ...","[0.00068, 0.00277, 0.00381, 0.00512, 0.00633, ...","[4e-06, 6.5e-05, 0.000188, 0.000423, 0.000712,...","[1.8e-05, 0.000131, 0.000289, 0.000515, 0.0009..."
1,AircraftBuoyant,./features/AircraftBuoyant\m1341,-0.458655,-0.605102,-0.021151,-0.691012,-1.082039,-0.191202,-1.437485,"[0.004708, 0.010527, 0.014929, 0.01883, 0.0220...","[0.0, 0.0024, 0.0046, 0.0049, 0.0032, 0.0064, ...","[0.00058, 0.0018, 0.00276, 0.00355, 0.00449, 0...","[8e-06, 3.7e-05, 0.000104, 0.000253, 0.000482,...","[2e-05, 9.4e-05, 0.000278, 0.000535, 0.000875,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,WheelChair,./features/WheelChair\D00687,-0.257985,-0.136291,-0.021151,-0.318480,-0.355524,0.236439,-0.413690,"[0.010272, 0.014989, 0.015033, 0.013491, 0.013...","[0.0, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, ...","[0.00213, 0.00465, 0.00622, 0.00658, 0.00785, ...","[4.5e-05, 0.000309, 0.000829, 0.001534, 0.0022...","[5.6e-05, 0.000388, 0.000926, 0.001629, 0.0025..."
1,WheelChair,./features/WheelChair\D00720,0.415856,-0.265226,-0.021151,-0.563298,0.443687,0.331357,0.558552,"[0.00313, 0.006494, 0.008935, 0.010701, 0.0120...","[0.0, 0.0001, 0.0001, 0.0001, 0.0001, 0.0003, ...","[0.00059, 0.00119, 0.00229, 0.0039, 0.0037, 0....","[1e-05, 5.1e-05, 0.000166, 0.000381, 0.000644,...","[2.9e-05, 0.000155, 0.000435, 0.000801, 0.0013..."
1,WheelChair,./features/WheelChair\D00782,0.330954,-0.270341,-0.021151,-0.133080,0.251680,0.460616,0.345153,"[0.001609, 0.002788, 0.003763, 0.004353, 0.004...","[0.0, 0.0002, 0.0001, 0.0001, 0.0001, 0.0001, ...","[0.00031, 0.00155, 0.00234, 0.00289, 0.00334, ...","[7e-06, 4.4e-05, 0.00016, 0.00035, 0.000593, 0...","[3.3e-05, 0.000174, 0.000441, 0.000865, 0.0014..."
1,WheelChair,./features/WheelChair\D00821,0.315740,-0.136219,-0.021151,-0.316797,0.369517,0.233050,0.444380,"[0.010325, 0.015111, 0.014943, 0.013719, 0.012...","[0.0, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, ...","[0.00276, 0.00394, 0.00617, 0.00705, 0.00779, ...","[5.5e-05, 0.000291, 0.000837, 0.001494, 0.0021...","[6.4e-05, 0.000409, 0.000957, 0.001821, 0.0026..."


In [5]:
def w_dist(a,b):
    return wasserstein_distance(a,b)*1000

In [6]:
valdf = df[['A3', 'D1', 'D2', 'D3', 'D4']]

In [68]:
newobj = df.iloc()[1300][['A3', 'D1', 'D2', 'D3', 'D4']]
newobj

A3    [0.005445, 0.012587, 0.016992, 0.020727, 0.023...
D1    [0.0, 0.0006, 0.0032, 0.0038, 0.0045, 0.0036, ...
D2    [0.00047, 0.0017, 0.0029, 0.00443, 0.00618, 0....
D3    [2.4e-05, 0.000276, 0.000935, 0.002035, 0.0036...
D4    [5e-05, 0.000375, 0.00089, 0.001752, 0.002815,...
Name: 1, dtype: object

In [69]:
w1=1;w2=1;w3=1;w4=1;w5=1
weights = np.array([w1,w2,w3,w4,w5])

In [70]:
weighted_sums = np.array([])
for index, row in valdf.iterrows():
        # print(valdf.columns[0])
        values = np.array([])
        for col in valdf.columns:
            dist = w_dist(newobj[col],row[col])
            # dist = get_emd(newobj[col],row[col])

            values = np.append(values, dist)
        weighted_sum = np.dot(values, weights)
        weighted_sums = np.append(weighted_sums, weighted_sum)
        # print(weighted_sum)

In [71]:
lowest_indices = np.argsort(weighted_sums)[:10]

lowest_values = weighted_sums[lowest_indices]

print("5 Lowest Values:", lowest_values)
print("Indices of the 5 Lowest Values:", lowest_indices)


5 Lowest Values: [0.         2.11851115 2.97657638 3.08118894 3.20097564 3.38957091
 3.42135312 3.63629361 3.72862602 3.84290286]
Indices of the 5 Lowest Values: [1300 1318 1309 1351 1364 1335  918 1310 1284 1361]


In [72]:
df.iloc[lowest_indices]["class"]

1           Jet
1           Jet
1           Jet
1           Jet
1           Jet
1           Jet
1    Helicopter
1           Jet
1           Jet
1           Jet
Name: class, dtype: object

In [ ]:
df[df["class"]=="Vase"]

In [10]:
df[df.apply(lambda x: x.isnull().any(), axis=1)]


,class,path,V,S,c,D,R,E,C,A3,D1,D2,D3,D4


In [11]:
def has_nan_in_list(row):
    return any(pd.isna(x) for x in row['D1'])
df_has_nan = df.apply(has_nan_in_list, axis=1)
df[df_has_nan]

,class,path,V,S,c,D,R,E,C,A3,D1,D2,D3,D4


In [65]:
import sys
sys.path.append('../preprocessing')
from viz import viz_mesh

In [85]:
viz_mesh("../resampledPML//Chess\m1591.obj"	)

In [75]:
def give_random_triplets(input=range(10), samples=10):
    return np.random.choice(input, size=(samples, 3), replace=True)

random_triplets = give_random_triplets(input=range(1000), samples=1000000)
random_triplets

array([[756, 820, 553],
       [ 86, 287,  27],
       [226, 402, 564],
       ...,
       [355, 783, 267],
       [586,  81, 968],
       [387, 647, 719]])

In [76]:
equal_indices = np.where((random_triplets[:, 0] == random_triplets[:, 1]) | (random_triplets[:, 1] == random_triplets[:, 2]) | (random_triplets[:, 0] == random_triplets[:, 2]))
equal_indices[0]

array([   454,    519,    583, ..., 999291, 999627, 999783], dtype=int64)

In [77]:
random_triplets[equal_indices] = give_random_triplets(samples=random_triplets[equal_indices].shape[0] )


In [194]:
%%timeit
random_triplets = give_random_triplets(input=range(7000), samples=1000000)
equal_indices = np.where((random_triplets[:, 0] == random_triplets[:, 1]) | (random_triplets[:, 1] == random_triplets[:, 2]) | (random_triplets[:, 0] == random_triplets[:, 2]))
while equal_indices[0].size > 0:
    random_triplets[equal_indices] = give_random_triplets(samples=random_triplets[equal_indices].shape[0] )
    equal_indices = np.where((random_triplets[:, 0] == random_triplets[:, 1]) | (random_triplets[:, 1] == random_triplets[:, 2]) | (random_triplets[:, 0] == random_triplets[:, 2]))

random_triplets


58.2 ms ± 4.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Accuracy measures

In [196]:
def get_top_k_results(newobj, k=10):
    w1=1;w2=1;w3=1;w4=4;w5=3
    weights = np.array([w1,w2,w3,w4,w5])
    weighted_sums = np.array([])
    for index, row in valdf.iterrows():
        # print(valdf.columns[0])
        values = np.array([])
        for col in valdf.columns:
            dist = w_dist(newobj[col],row[col])
            # dist = get_emd(newobj[col],row[col])

            values = np.append(values, dist)
        weighted_sum = np.dot(values, weights)
        weighted_sums = np.append(weighted_sums, weighted_sum)
        # print(weighted_sum)
        lowest_indices = np.argsort(weighted_sums)[:k+1]

        lowest_values = weighted_sums[lowest_indices]

    print("5 Lowest Values:", lowest_values)
    print("Indices of the 5 Lowest Values:", lowest_indices)
    return lowest_indices, lowest_values


In [82]:

newobj = df.iloc()[[1300,1]][['A3', 'D1', 'D2', 'D3', 'D4']]
newobj

,A3,D1,D2,D3,D4
1,"[0.005445, 0.012587, 0.016992, 0.020727, 0.023...","[0.0, 0.0006, 0.0032, 0.0038, 0.0045, 0.0036, ...","[0.00047, 0.0017, 0.0029, 0.00443, 0.00618, 0....","[2.4e-05, 0.000276, 0.000935, 0.002035, 0.0036...","[5e-05, 0.000375, 0.00089, 0.001752, 0.002815,..."
1,"[0.000905, 0.002401, 0.003699, 0.004898, 0.006...","[0.0, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, ...","[0.00013, 0.00038, 0.0006, 0.001, 0.00152, 0.0...","[3e-06, 1.6e-05, 4.9e-05, 0.000101, 0.000188, ...","[8e-06, 6.2e-05, 0.00014, 0.000303, 0.00046, 0..."


In [ ]:
df.iloc[lowest_indices]["class"]

1           Jet
1           Jet
1           Jet
1           Jet
1           Jet
1           Jet
1    Helicopter
1           Jet
1           Jet
1           Jet
Name: class, dtype: object

In [158]:
import pandas as pd
import random

# Create an empty dictionary to store the random indices
random_indices_dict = {}
indices_arr = np.array([],dtype=int)
# Group the DataFrame by the "class" column
grouped = df.groupby('class')

# Iterate over each group and select two different random indices for each
for name, group in grouped:
    indices = group.index.tolist()
    newvals = random.sample(indices, min(2, len(indices)))
    random_indices_dict[name] = newvals
    indices_arr = np.append(indices_arr,newvals)

# Display the random indices for each distinct class
print(random_indices_dict)
print(indices_arr)


{'AircraftBuoyant': [12, 15], 'Apartment': [25, 36], 'AquaticAnimal': [52, 44], 'Bed': [88, 84], 'Bicycle': [101, 105], 'Biplane': [147, 150], 'Bird': [199, 173], 'Bookset': [211, 218], 'Bottle': [234, 239], 'BuildingNonResidential': [271, 279], 'Bus': [314, 318], 'Car': [390, 377], 'Cellphone': [430, 426], 'Chess': [453, 457], 'City': [484, 495], 'ClassicPiano': [500, 519], 'Computer': [548, 536], 'ComputerKeyboard': [567, 559], 'Cup': [598, 590], 'DeskLamp': [622, 617], 'DeskPhone': [636, 639], 'Door': [665, 688], 'Drum': [711, 714], 'Fish': [725, 717], 'FloorLamp': [759, 744], 'Glasses': [769, 777], 'Guitar': [788, 797], 'Gun': [860, 819], 'Hand': [868, 897], 'Hat': [912, 903], 'Helicopter': [932, 942], 'House': [991, 985], 'HumanHead': [1059, 1019], 'Humanoid': [1163, 1185], 'Insect': [1238, 1233], 'Jet': [1319, 1376], 'Knife': [1430, 1423], 'MilitaryVehicle': [1456, 1444], 'Monitor': [1472, 1473], 'Monoplane': [1492, 1485], 'Motorcycle': [1505, 1502], 'Mug': [1524, 1538], 'MultiSe

In [195]:
get_top_k_results(newobj, k = 10)

5 Lowest Values: [0.         2.6673     3.0845001  3.16317584 3.19272594 3.20122
 3.26587334 3.29074614 3.32684614 3.39054438 3.39365327]
Indices of the 5 Lowest Values: [2461 2458 2334 1982 1984  876 1974  897 1633 1555 1590]


(array([2461, 2458, 2334, 1982, 1984,  876, 1974,  897, 1633, 1555, 1590],
       dtype=int64),
 array([0.        , 2.6673    , 3.0845001 , 3.16317584, 3.19272594,
        3.20122   , 3.26587334, 3.29074614, 3.32684614, 3.39054438,
        3.39365327]))

In [197]:
totalcorrect = 0
total = 0
for i in indices_arr:
    newobj = df.iloc()[i][['A3', 'D1', 'D2', 'D3', 'D4']]
    lowest_indices, lowest_values = get_top_k_results(newobj, k = 10)
    y = df.iloc[lowest_indices[0]]["class"]
    correct = 0
    for ind in lowest_indices[1:]:
        if df.iloc[ind]["class"] == y:
            correct += 1 
    totalcorrect += correct
    total += len(lowest_indices[1:])

5 Lowest Values: [0.         5.18361369 5.2050163  5.97452358 6.74639075 6.93112399
 6.95369467 6.96991369 7.03672712 7.07421196 7.0825094 ]
Indices of the 5 Lowest Values: [  12 2042  725  723  729  732 1731  163  935  717 2056]
5 Lowest Values: [0.         4.15291109 4.89312307 5.65893149 6.02562059 6.07849089
 6.29260178 6.52163327 6.55868694 6.59568139 6.7134299 ]
Indices of the 5 Lowest Values: [  15 1911 1810 1527  256 1060  240 1825  238 1675  900]
5 Lowest Values: [ 0.          5.40129062  6.44156565  7.70852     7.92426098  9.02860056
  9.22287861  9.76678    10.17656106 10.32842728 10.46266832]
Indices of the 5 Lowest Values: [  25 1623 1789  983  339  970  338  329  424  336 2290]
5 Lowest Values: [0.         6.95423324 7.25018683 7.4060845  7.72385815 8.00412056
 8.10941183 8.28475842 8.3206828  8.35741764 8.43298064]
Indices of the 5 Lowest Values: [  36  191  263 2146 1721 1730 2058 1409 2306  964 1775]
5 Lowest Values: [0.         5.19211893 5.24209868 5.50762385 5.55940

In [199]:
totalcorrect, total, totalcorrect/ total

(209, 1380, 0.15144927536231884)

In [192]:
totalcorrect, total, totalcorrect/ total

(238, 1380, 0.17246376811594202)

In [188]:
df.iloc[lowest_indices]

,class,path,V,S,c,D,R,E,C,A3,D1,D2,D3,D4
100,Bicycle,./features/Bicycle\D00462,0.015860,-0.558199,-0.021151,-0.530023,-0.014967,-0.675910,-0.021856,"[0.013414, 0.021464, 0.02267, 0.022876, 0.0231...","[0.0, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, ...","[0.00175, 0.00451, 0.00605, 0.00585, 0.00708, ...","[0.00012, 0.000591, 0.001199, 0.001893, 0.0027...","[0.000103, 0.000612, 0.001531, 0.002789, 0.004..."
102,Bicycle,./features/Bicycle\D00536,-0.003958,-0.594845,-0.021151,-0.509049,-0.024217,-0.458325,-0.038430,"[0.011727, 0.017246, 0.021203, 0.021615, 0.021...","[0.0, 0.0001, 0.0001, 0.0001, 0.0002, 0.0001, ...","[0.00227, 0.0065, 0.00635, 0.00573, 0.00592, 0...","[0.00015, 0.000766, 0.001386, 0.002328, 0.0032...","[0.000144, 0.000953, 0.002239, 0.003976, 0.006..."
1465,Monitor,./features/Monitor\D00391,-0.179488,1.204441,-0.021151,0.852796,-0.389738,-0.702152,-0.545376,"[0.009992, 0.01945, 0.021982, 0.022273, 0.0224...","[0.0, 0.001, 0.0011, 0.0019, 0.001, 0.0011, 0....","[0.00073, 0.002, 0.00271, 0.0033, 0.00487, 0.0...","[3.2e-05, 0.000166, 0.000468, 0.000992, 0.0016...","[0.000744, 0.000818, 0.001055, 0.001956, 0.002..."
108,Bicycle,./features/Bicycle\D00750,0.100926,-0.272764,-0.021151,-0.372126,0.098439,-0.604313,0.151024,"[0.009781, 0.019754, 0.023327, 0.023564, 0.023...","[0.0, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, ...","[0.00054, 0.00234, 0.00401, 0.00534, 0.00647, ...","[1.4e-05, 0.000117, 0.000433, 0.001004, 0.0017...","[6.6e-05, 0.000416, 0.001103, 0.001944, 0.0033..."
114,Bicycle,./features/Bicycle\D00974,-0.024465,-0.548177,-0.021151,-0.498631,-0.117300,-0.688648,-0.187253,"[0.011226, 0.021829, 0.026766, 0.02798, 0.0269...","[0.0, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, ...","[0.00158, 0.00473, 0.00715, 0.00809, 0.00925, ...","[3.7e-05, 0.000395, 0.000988, 0.001959, 0.0030...","[7.6e-05, 0.000494, 0.001341, 0.002521, 0.0039..."
96,Bicycle,./features/Bicycle\D00033,0.011966,-0.672358,-0.021151,-0.687026,-0.021327,-0.704064,-0.034429,"[0.012993, 0.022425, 0.024989, 0.025044, 0.026...","[0.0, 0.0001, 0.0001, 0.0004, 0.0002, 0.0003, ...","[0.00169, 0.00331, 0.00459, 0.00462, 0.00762, ...","[8.7e-05, 0.00057, 0.001067, 0.001509, 0.00225...","[0.000108, 0.000687, 0.001729, 0.00331, 0.0050..."
115,Bicycle,./features/Bicycle\m1472,0.001595,-0.627271,-0.021151,-0.647436,-0.019638,-0.642163,-0.038314,"[0.011378, 0.022022, 0.024566, 0.024305, 0.023...","[0.0, 0.0001, 0.0001, 0.0001, 0.0012, 0.0005, ...","[0.00138, 0.00399, 0.00619, 0.00623, 0.00645, ...","[2.6e-05, 0.000247, 0.000791, 0.00152, 0.00252...","[0.000117, 0.000755, 0.001949, 0.003755, 0.005..."
106,Bicycle,./features/Bicycle\D00621,0.045538,-0.656544,-0.021151,-0.634887,0.044704,-0.723747,0.060357,"[0.016849, 0.026814, 0.029573, 0.028311, 0.027...","[0.0, 0.0001, 0.0001, 0.0001, 0.0005, 0.0009, ...","[0.00165, 0.00388, 0.00539, 0.00654, 0.00796, ...","[0.000116, 0.000666, 0.001466, 0.00253, 0.0038...","[0.000143, 0.001057, 0.002642, 0.004558, 0.007..."
104,Bicycle,./features/Bicycle\D00558,-0.014542,-0.538586,-0.021151,-0.656887,-0.077964,-0.676072,-0.103970,"[0.009977, 0.019977, 0.023832, 0.024862, 0.024...","[0.0, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, ...","[0.00107, 0.00332, 0.00505, 0.006, 0.00729, 0....","[3.2e-05, 0.000223, 0.00072, 0.001249, 0.00209...","[7e-05, 0.00045, 0.001221, 0.002446, 0.00398, ..."
99,Bicycle,./features/Bicycle\D00078,0.067563,-0.551459,-0.021151,-0.280044,0.052743,-0.578388,0.088031,"[0.008261, 0.015829, 0.019215, 0.021177, 0.021...","[0.0, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, ...","[0.00165, 0.00433, 0.00711, 0.00694, 0.00803, ...","[4.8e-05, 0.000358, 0.000952, 0.001911, 0.0029...","[7.2e-05, 0.000499, 0.001296, 0.002559, 0.0040..."


In [11]:
df2 = df

In [21]:
min(df[['A3', 'D1', 'D2', 'D3', 'D4']].applymap(len).min())

12

In [29]:
list_lengths = df[['A3', 'D1', 'D2', 'D3', 'D4']].applymap(len)

# Find the row indices with the maximum and minimum list lengths
row_with_max_length = list_lengths.sum(axis=1).idxmax()
row_with_min_length = list_lengths.sum(axis=1).idxmin()

# Get the corresponding rows from the original DataFrame
row_with_max_length_data = df.iloc[row_with_max_length]
row_with_min_length_data = df.iloc[row_with_min_length]

# Find the columns with the maximum and minimum list lengths
max_length_columns = list_lengths.loc[row_with_max_length].idxmax()
min_length_columns = list_lengths.loc[row_with_min_length].idxmin()

# Get the respective lengths of the maximum and minimum list lengths
max_length = list_lengths.loc[row_with_max_length, max_length_columns]
min_length = list_lengths.loc[row_with_min_length, min_length_columns]

print("Row with the highest list length (Row Index:", row_with_max_length, ")")
print(row_with_max_length_data)
print("Columns with the highest list length:", max_length_columns)
print("Length:", max_length)

print("\nRow with the lowest list length (Row Index:", row_with_min_length, ")")
print(row_with_min_length_data)
print("Columns with the lowest list length:", min_length_columns)
print("Length:", min_length)

Row with the highest list length (Row Index: 126 )
class                                              Biplane
path                             ./features/Biplane\D00276
V                                                 0.317499
S                                                -0.242264
c                                                -0.021151
D                                                -0.455042
R                                                 0.235541
E                                                 0.305287
C                                                 0.297418
A3       [0.004609, 0.009604, 0.012367, 0.01425, 0.0157...
D1       [0.0, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, ...
D2       [0.0004, 0.00167, 0.00304, 0.00375, 0.00485, 0...
D3       [1.5e-05, 0.000133, 0.000491, 0.001019, 0.0017...
D4       [7.7e-05, 0.000313, 0.000951, 0.001741, 0.0027...
Name: 126, dtype: object
Columns with the highest list length: D1
Length: 101

Row with the lowest list length (Row Index: 

In [37]:
df[df["path"] == "./features/Car\m1510"]["D1"].iloc[0]

array([0.000e+00, 2.040e-02, 7.110e-02, 1.141e-01, 1.147e-01, 1.297e-01,
       1.772e-01, 1.856e-01, 1.369e-01, 3.200e-02, 1.340e-02, 3.800e-03,
       1.000e-04, 1.000e-04, 1.000e-04, 1.000e-04, 1.000e-04, 1.000e-04,
       1.000e-04, 1.000e-04, 1.000e-04, 1.000e-04, 1.000e-04])

In [16]:
# Determine the maximum number of elements in any list
max_length = 100#max(df.applymap(len).max())

# Expand lists to have the same length
for col in df.columns:
    df[col] = df[col].apply(lambda x: x + [None] * (max_length - len(x)))

# Create new columns for each element in the lists
for i in range(max_length):
    df[f"A3_{i+1}"] = df['A3'].apply(lambda x: x[i])
    df[f"D1_{i+1}"] = df['D1'].apply(lambda x: x[i])
    df[f"D2_{i+1}"] = df['D2'].apply(lambda x: x[i])
    df[f"D3_{i+1}"] = df['D3'].apply(lambda x: x[i])
    df[f"D4_{i+1}"] = df['D4'].apply(lambda x: x[i])

# Drop the original columns with lists
df = df.drop(columns=['A3', 'D1', 'D2', 'D3', 'D4'])

TypeError: can only concatenate str (not "list") to str